# Chapter 4 - Bedrock APIs: Customer Conversation Summarization with Amazon Bedrock

## Overview
This notebook demonstrates how to generate customer conversation summaries using Amazon Bedrock APIs. We'll explore how to use foundation models to enhance service quality through automated conversation analysis and summarization.

## Introduction
This notebook demonstrates how to interact with Amazon Bedrock APIs. We'll explore how to list available foundation models, generate text using the standard API, and use the newer Converse API for multi-turn conversations.In this example, you are going to evaluate a scenario of generating customer conversation summaries using generative AI to enhance service quality.

## Prerequisites
- AWS account with Amazon Bedrock access
- Access to Claude 3 models
- Basic understanding of AWS SDK for Python (boto3)

## Setup

### Install Required Dependencies

In [ ]:
# Install the AWS SDK for Python
%pip install -U --no-cache-dir boto3

### Import Libraries

In [ ]:
# Import required libraries for JSON handling, AWS SDK, and error handling
import json
import boto3
import botocore

# Initialize Bedrock runtime client for invoking foundation models
boto3_bedrock = boto3.client('bedrock-runtime')

## Summarizing Text with Amazon Bedrock

To learn details of API requests to Amazon Bedrock, this notebook introduces how to create API requests and send them via Boto3.

### Request Syntax of InvokeModel in Boto3

We use the `InvokeModel` API for sending requests to a foundation model. Here we'll demonstrate with Amazon Nova Micro and Claude 3.5 Haiku models. Inference parameters depend on the model you are using.

**Amazon Nova Micro Inference Parameters:**

- **maxTokenCount**: Configures the max number of tokens to use in the generated response
- **stopSequences**: Used to make the model stop at a desired point, such as the end of a sentence or list
- **temperature**: Modulates the probability density function for next tokens (0-1, lower = more deterministic)
- **topP**: Controls token choices based on probability of potential choices (below 1.0 = more stable)

```python
response = bedrock.invoke_model(
    body={
        "inputText": "<<place your input text here>>",
        "textGenerationConfig": {
            "maxTokenCount": 4096,
            "stopSequences": [],
            "temperature": 0,
            "topP": 1
        }
    },
    modelId="amazon.nova-micro-v1:0",
    accept=accept,
    contentType=contentType
)
```

### Writing Prompt with Text to be Summarized

In this notebook, you can use any short text whose tokens are less than the maximum token limit of a foundation model. The prompt starts with an instruction: *"Please provide a summary of the following text."*

In [ ]:
# Define the prompt with customer service conversation to be summarized
prompt = """
Please provide a summary of the following text. Do not add any information that is not mentioned in the text below.

<text>
[Agent] Hi, thank you for calling Best Internet company. This is Collin, how may I help. You?
[Customer] Hi Collin. So, I'm calling because I have been overcharged, um, my subscription to you guys is supposed to be a flat rate of 50 dollars per month but for some reason I am seeing 75 dollars on my paper bill for the month of March. So, I don't know what's going on. I would really appreciate an explanation on this.
[Agent] Sure, I'll be happy to pull up your account now and check the billing details for you. Can I have your account number together with your first and last name?
[Customer] Sure it's one two three four five and my full name is Jason Walls.
[Agent] Williams. Thank you, Jason. Just give me a few seconds to pull up your account. It was for the month of March this year, right? March 2024?
[Customer] That's right.
[Agent] Okay let me see that. Okay I have your account pulled up now and I'm only seeing a 50 charge for the month of March this year. Could you recheck your bill and review the amount?
[Customer] No. That can't be right. I am literally holding my bill right now and it says March and it says 75 dollars in big bold letters. So, may the charge be moved to Feb or April? I don't know. All I know is that you're overcharging me and it's written on my paper bill.
[Agent] Yes, Jason, I also checked your other month's charges this year from January to March 2024 and all of these months you were only charged 50 dollars. Let me just check further though your last year. Oh, although I am seeing 75 dollars charge here but this was way back last year on March 2023. Would you mind checking the year of the billing paper you're holding right now?
[Customer] Oh My Goodness! Collin, I'm so, so, sorry to have wasted your time. I really thought this was a recent bill.
[Agent] No, don't worry about it, Jason. You're fine.
[Customer] This bill was literally on my table this morning. I must have dropped it when I was throwing garbage this morning. So embarrassing, my bad.
[Agent] Oh, no problem. It happens, I mean it happens to all of us you know.
[Laughter]
[Customer] I know, right, anyway Collin I won't waste any more of your time, and thank you so much for your patience.
[Agent] You're welcome Jason. Is there anything else that I can help you with today?
[Customer] No, that's all Collin. Have a good one.
[Agent] Have a good one Jason. Thank you for calling Best Internet. Bye.
[Customer] Bye, Collin.
</text>

"""

In [ ]:
# Create JSON request body for Nova Micro with correct parameters
body = json.dumps({
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": prompt
                }
            ]
        }
    ],
    "inferenceConfig": {
        "maxTokens": 4096,
        "temperature": 0,
        "topP": 1
    }
})

## Invoke Foundation Model via Boto3

Here we send the API request to Amazon Bedrock by specifying request parameters `modelId`, `accept`, and `contentType`. Following the prompt, the foundation model in Amazon Bedrock summarizes the text.

In [ ]:
# Set model parameters and invoke Nova Micro to generate conversation summary
modelId = 'amazon.nova-micro-v1:0'
accept = 'application/json'
contentType = 'application/json'

try:
    # Call Bedrock API to invoke the Nova Micro model
    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    # Extract and print the generated summary
    print(response_body['output']['message']['content'][0]['text'])
    
except botocore.exceptions.ClientError as error:
    # Handle access denied errors with helpful troubleshooting links
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

## Using Claude 3.5 Haiku Model

Let's try to use Claude 3.5 Haiku model for the same scenario to compare results.

In [ ]:
# Create JSON request body for Claude 3.5 Haiku with different parameter structure
body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1000,
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt
          }
        ]
      }
    ],
    "temperature":0.5,
    "top_k":250,
    "top_p":0.5,
    "stop_sequences":[]
})

In [ ]:
# Invoke Claude 3.5 Haiku model using inference profile
modelId = 'us.anthropic.claude-3-5-haiku-20241022-v1:0'
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

# Print Claude's response
print(response_body.get('content')[0]['text'])

## Conclusion

You have now experimented with using the boto3 SDK to invoke Amazon Bedrock APIs. Using this API, you have seen the use case of generating a summary of call transcripts with different foundation models:

- **Amazon Nova Micro**: Provides fast, cost-effective summaries with good accuracy
- **Claude 3.5 Haiku**: Offers quick, efficient summaries with excellent performance

Both models successfully identified the key points of the customer service conversation and provided accurate summaries without adding information not present in the original text.